<a href="https://colab.research.google.com/github/MailSuesarn/MailSuesarn.github.io/blob/master/Demonstrate_Class_Activation_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📁 *This dataset is part of the MVTec dataset* 📁

Paul Bergmann, Michael Fauser, David Sattlegger, Carsten Steger. MVTec AD - A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection; in: IEEE Conference on Computer Vision and Pattern Recognition (CVPR), June 2019

[MVTec Dataset](https://www.mvtec.com/company/research/datasets/mvtec-ad)

This Colab is a part of the Super AI Engineer article
> Author: Suesarn Wilainuch (22p21c0153)

> Home: EXP

🔴🟢🔵 let's get started! 🔴🟢🔵

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import Progbar
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB3

import os
from tqdm import tqdm
from numpy import load
import cv2

# Load dataset (in memory (RAM))

In [ ]:
def load_data(filename):
    data = load(filename)
    X, y = data['arr_0'], data['arr_1']
    return X, y

# dataset path
X, y = load_data('/gdrive/My Drive/Tensorflow2_tutorial/Medium&Video/hazelnut_dataset.npz')

print(X.shape, y.shape)

(170, 300, 300, 3) (170,)


# Split train, test and prepare training data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=153, stratify=y)

trainset = tf.data.Dataset.from_tensor_slices((X_train.astype('float32'), y_train)).shuffle(len(X_train))

# Create model

In [ ]:
def create_model(image_shape):

    in_image = tf.keras.layers.Input(shape=image_shape)

    model = EfficientNetB3(include_top=False, input_tensor=in_image, weights="imagenet")

    model.trainable = True

    f = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    top_dropout_rate = 0.4
    f = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(f)
    outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred")(f) # change activation and number of output node 

    # Compile
    model = tf.keras.Model(in_image, outputs)
    opt = tf.keras.optimizers.Adam(lr=0.0001)  
    model.compile(loss=['sparse_categorical_crossentropy'], optimizer=opt, metrics=['accuracy']) # change loss function
    model.summary()
    return model

In [ ]:
IMAGE_SHAPE = (300, 300, 3)
model = create_model(IMAGE_SHAPE)

# function for create CAM (select activation function softmax or sigmoid) 

In [ ]:
def create_CAM(input_image, model, activation='softmax'):
    CAM_list = list()
    n = input_image.shape[0]

    last_conv = tf.keras.Model(model.input, model.get_layer("top_activation").output) # specified by layer name
    weights = model.layers[-1].get_weights()[0]  # weight at output class
    feature_map = last_conv.predict(input_image) # (batch, h, w, c)
    pred_prob = model.predict(input_image)
    pred = np.squeeze(np.argmax(pred_prob, axis=-1))

    ch = feature_map.shape[-1] # channel
    hw = feature_map.shape[1]  # height = width
    N = hw * hw  # N = height x width
    feature_map = feature_map.reshape(-1, N, ch) # (bs, h, w, c) --> (bs, N, c)

    for i in tqdm(range(n)):
      if activation == 'sigmoid':
        weights_class = weights[:, 0] # weight of activate class in this case we have only one (sigmoid)
      else:
        # pred[i] = 1 - pred[i] # toggle for plot deactivate class
        weights_class = weights[:, pred[i]] # select weight that correspond to activate class 

      feature = feature_map[i]
      CAM = np.dot(feature, weights_class).reshape((hw, hw, 1))  # (h, w, 1)
      CAM_list.append(CAM)
  
    return np.asarray(CAM_list)

# Plot CAM

In [ ]:
from IPython import display

def plot_CAM(input_image, CAMs, pred):
  rows = 4
  cols = 4
  axes = []
  fig = plt.figure()

  display.clear_output(wait=True)
  for i in range(rows*cols):
      axes.append(fig.add_subplot(rows, cols, i+1))
      # label
      label = "Good" if y_test[i] == 0 else "Bad" 

      # predicted
      class_name = "Good" if pred[i] == 0 else "Bad" 

      if class_name == label:
        subplot_title=(f"{class_name}|{label}")
        axes[-1].set_title(subplot_title, color='k')
      else:
        subplot_title=(f"{class_name}|{label}")
        axes[-1].set_title(subplot_title, color='r')

      plt.axis('off')  
      plt.imshow(input_image[i].astype('uint8'))
      CAM = CAMs[i]
      CAM = cv2.resize(CAM, (X.shape[1], X.shape[1]))
      plt.imshow(CAM, cmap='jet', alpha=0.3)
  fig.tight_layout()    
  plt.savefig(str(epoch+1) + '.png')
  plt.show()

# Training & create CAM on each epoch (Use train_on_batch method)

In [ ]:
batch_size = 16
epochs = 100

In [ ]:
for epoch in tqdm(range(epochs)):

    # Loop on each batch of training
    for idX, (X_batch, y_batch) in enumerate(trainset.batch(batch_size)):
        
      loss, acc = model.train_on_batch(X_batch, y_batch)
        
    print(f"loss: {loss}, acc: {acc}")

    # plot CAM and image for visualization on each epoch
    CAMs = create_CAM(X_test, model, activation='sigmoid') 

    pred_prob = model.predict(X_test)

    pred = np.argmax(pred_prob, axis=-1) # for softmax 
    # pred = np.round(pred_prob, 1).astype(np.int32) # for sigmoid

    plot_CAM(X_test, CAMs, pred)     

# Training & create CAM on each epoch (Use custom training loop method)

Define loss function

In [ ]:
sparse_cat_cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_fn(y_true, y_pred):
    loss = sparse_cat_cross_entropy(y_true, y_pred)
    return loss

Define the optimizer and prepare the metrics 

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)

acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

metrics_names = ['train_loss', 'train_acc']
num_training_samples = X_train.shape[0]
epochs = 100
batch_size = 16

Define training function

In [ ]:
@tf.function
def train_step(X, y):
    with tf.GradientTape() as tape:
        pred = model(X, training=True)

        loss = loss_fn(y, pred)

    gradients_of_model = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients_of_model, model.trainable_variables))
    acc_metric.update_state(y, pred)
    return loss

Define training loop

❗ if you want to use sigmoid, Don't forget to change loss function and accuracy metric

In [ ]:
from IPython import display

for epoch in range(epochs):

    print("\nEpoch {}/{}".format(epoch + 1, epochs))

    progBar = Progbar(num_training_samples, stateful_metrics=metrics_names)

    # Loop on each batch of training
    for idX, (X_batch, y_batch) in enumerate(trainset.batch(batch_size)):
        train_loss = train_step(X_batch, y_batch)
        train_acc = acc_metric.result()
        acc_metric.reset_states()
        values = [('train_loss', train_loss), ('train_acc', train_acc)]
        progBar.update(idX * batch_size, values=values)
    
    # plot CAM and image for visualization on each epoch
    CAMs = create_CAM(X_test, model, activation='softmax')

    pred_prob = model.predict(X_test)
    pred = np.argmax(pred_prob, axis=-1) # for softmax 
    plot_CAM(X_test, CAMs, pred)

# Create .gif file for visualization CAM on each epoch

In [ ]:
import imageio
anim_file = 'pretrained_visualization_CAM.gif'

with imageio.get_writer(anim_file, mode='i') as writer:
  for i in tqdm(range(epochs)):
    filename = str(i+1) + '.png'
    image = imageio.imread(filename)
    writer.append_data(image)
  writer.append_data(image)

# Evaluate 

In [ ]:

_, acc = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy:", acc*100)